In [ ]:
'''
安裝指令: pip install --user plotly_express==0.4.0
'''
import plotly.express as px

import datetime
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from finlab.data import Data
from finlab.plot_candles import plot_candles

def percentage(target, today):
    """ 計算百分比
    Args:
        target: 目標價格
        today: 目前價格
    Return:
        百分比
    """
    return 100 * (target / today - 1)


def longprofit(buy_price, sell_price, discount=0.28):
    
    fee = 0.001425 * discount
    tax = 0.003

    買入手續費 = buy_price * 1000 * fee
    買入手續費 = 20 if 買入手續費 < 20 else int(買入手續費)
    賣出手續費 = sell_price * 1000 * fee
    賣出手續費 = 20 if 賣出手續費 < 20 else int(賣出手續費)
    賣出證交稅 = (sell_price * 1000 * tax).round(0)

    獲利 = 100 * ((sell_price * 1000 - 賣出手續費 - 賣出證交稅) / (buy_price * 1000 + 買入手續費) - 1)
    return 獲利


def data_mapping2daily(partial, compelte, fill=True):
    
    sid = compelte.columns & partial.columns
    
    frame = pd.DataFrame(index=compelte.index, columns=sid)
    newdata = pd.concat([partial[sid], frame], sort=True)
    newdata = newdata[~newdata.index.duplicated('first')]
    newdata = newdata.sort_index()
    # 是否要用前面的資料填滿
    if fill == True:
        newdata = newdata.fillna(method='ffill')
    # 保留共有的日期
    keep_dates = newdata.index.intersection(compelte.index)
    return newdata.loc[keep_dates, :]

def pick_stocks(date, stock_group, conditions, n=2):

    # 先過濾基本面
    sid = stock_group 
    satisfy = pd.Series(True, index=sid)

    for c in conditions:  
        # 取得stock_id的交集
        sid &= c.columns
        # 取得與選擇日期最接近的一筆資料
        satisfy &= c.loc[:date, sid].iloc[-1]

    psid = satisfy[satisfy]
    frame = 收盤價.loc[:date, psid.index].iloc[-n:]
    
    satisfy = pd.DataFrame(index=frame.index, columns=frame.columns, data=True)
    for c in conditions:    
        satisfy &= data_mapping2daily(c, frame)

    signal = satisfy.sum() == 1
    return signal[signal].index.tolist()


def 選股條件成立顯示(conditions, sid):
    print('條件數量: {}'.format(len(conditions)))
    i = 0
    for c in conditions:
        print('{}: {}'.format(i, conditions[i][sid].iloc[-1]))
        i = i + 1
                
def get_trend_up_date(date, sid):
    trigger = (ma20.loc[:date, sid] >= ma40.loc[:date, sid]) & (ma20.loc[:date, sid].shift(1) < ma40.loc[:date, sid].shift(1))
    trigger_date = trigger[trigger].index[-1].date()
    days = 收盤價.loc[trigger_date:date, sid].size - 1
    profit = round(percentage(最高價.loc[date, sid], ma5.loc[trigger_date, sid]), 2)
    return [trigger_date, days, profit]
    
    
def get_trend_down_date(date, sid):
    # 連續4天收盤價低於月均線
    condi1 = (收盤價.loc[date:, sid] < ma20.loc[date:, sid]).rolling(4).sum() >= 4
    # 月均線下彎
    condi2 = percentage(ma20.loc[date:, sid], ma20.loc[date:, sid].shift(1)) < 0
    
    trigger = condi1 & condi2
    trigger_date = trigger[trigger].index[0].date()
    trigger_days = 收盤價.loc[date:trigger_date, sid].size - 1
    profit = round(percentage(收盤價.loc[trigger_date, sid], 開盤價.loc[date:, sid].iloc[1]), 2)
    
    max_profit_date = 最高價.loc[date:trigger_date, sid].idxmax().date()
    max_profit_days = 收盤價.loc[date:max_profit_date, sid].size - 1
    max_profit = round(percentage(最高價.loc[max_profit_date, sid], 開盤價.loc[date:, sid].iloc[1]), 2)
    ma5_profit = round(percentage(ma5.loc[max_profit_date, sid], 開盤價.loc[date:, sid].iloc[1]), 2)
    
    return [trigger_date, trigger_days, profit, max_profit_date, max_profit_days, max_profit, ma5_profit]


def draw_finance(date, sid):
    
    s0 = 當月營收.loc[:date, sid].tail(24) / 1e5
    s1 = 近3月營收年增率.loc[:date, sid].tail(24)
    s2 = 近6月營收年增率.loc[:date, sid].tail(24)
    s3 = 近12月營收年增率.loc[:date, sid].tail(24)
    df = pd.concat([s0, s1, s2, s3], axis=1, join_axes=[s0.index])
    df.columns = ['月', 'k3', 'k6', 'k12']
    df_draw = df.reset_index()

    fig, ax = plt.subplots(1,2, figsize=(16,4))

    ax1 = plt.subplot(1,2,1)
    plt.title("月營收")

    width = 0.5
    df_draw['月'].plot(kind='bar', width=width, align='center', alpha=0.7)
    df_draw['k3'].plot(secondary_y=True, color='r', marker='o')
    df_draw['k6'].plot(secondary_y=True, color='y', marker='o')
    df_draw['k12'].plot(secondary_y=True, color='g', marker='o')
    ax1.set_xticklabels([d.strftime('%Y-%m') for d in df_draw['date']])
    plt.xlim([-width, len(df_draw['月'])-width])

    ax2 = plt.subplot(1,2,2)
    plt.title("毛利 vs 淨利")

    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m')) 
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=2)) 
    R105_營業毛利率.loc[:date, sid].tail(8).plot(marker='o')
    R108_稅後淨利率.loc[:date, sid].tail(8).plot(marker='o')
    plt.legend(['毛利', '淨利'])
    收盤價.loc[:date, sid].tail(500).plot(secondary_y=True)
    
    plt.gcf().autofmt_xdate(rotation=45) 
    plt.show()

def valuation_stocks(date, f_date, sids, pe_ratio=None):

    finance_date = date if date > f_date else f_date
    稅率 = 0.8
    減資比例 = round(股本.loc[:finance_date, sids].iloc[-2] / 股本.loc[:finance_date, sids].iloc[-3], 2)
    
    if pe_ratio == None:
        產業本益比 = pd.Series(index=sids, data=[get_category_pe(sid) for sid in sids])
    else:
        產業本益比 = pe_ratio
        
    最新當月營收 = 當月營收.loc[:finance_date, sids].iloc[-1]
    # 備註最新毛利率在2021.3.15已經公布，但資料庫時間是2021.3.31，因此最新毛利率透過手動輸入
    最新毛利率 = R105_營業毛利率.loc[:finance_date, sids].iloc[-1]
    最新營業費用 = T3300營業費用.loc[:finance_date, sids].iloc[-1]
    估計業外收入 = T3700營業外收入及支出.loc[:finance_date, sids].iloc[-1]
    上季稅後淨利 = T3970經常稅後淨利.loc[:finance_date, sids].iloc[-2]
    上季EPS = EPS.loc[:finance_date, sids].iloc[-2]

    預估季營收 = 最新當月營收 * 3
    預估毛利 = 預估季營收 * 最新毛利率 * 0.01
    預估營業利益 = 預估毛利 - 最新營業費用
    預估稅前淨利 = 預估營業利益 + 估計業外收入
    預估稅後淨利 = 預估稅前淨利 * 稅率
    比值 = 預估稅後淨利 / 上季稅後淨利
    預估季EPS = 比值 * 上季EPS
    預估年EPS = 預估季EPS * 4
    減資後EPS = 預估年EPS / 減資比例
    產業價值 = 減資後EPS * 產業本益比
    當日收盤價 = 收盤價.loc[date, sids]
    預估成長 = round(percentage(產業價值, 當日收盤價), 2)
    
    return 預估成長
#--------------------------------------------------------
data = Data()
data.warrning=True

# === 大盤指數 ===
指數開盤價 = data.get("指數開盤價")
指數最高價 = data.get("指數最高價")
指數最低價 = data.get("指數最低價")
指數收盤價 = data.get("指數收盤價")
指數成交值 = round(data.get("指數成交金額") / 1e8, 2)

指數收盤價ma5 = 指數收盤價.rolling(5).mean()
指數收盤價ma10 = 指數收盤價.rolling(10).mean()
指數收盤價ma20 = 指數收盤價.rolling(20).mean()
指數收盤價ma60 = 指數收盤價.rolling(60).mean()
指數成交值ma5 = 指數成交值.rolling(5).mean()
指數成交值ma20 = 指數成交值.rolling(20).mean()

# === 技術面指標資料 ===
開盤價 = data.get("開盤價").ffill()
最高價 = data.get("最高價").ffill()
最低價 = data.get("最低價").ffill()
收盤價 = data.get("收盤價").ffill()
成交值 = data.get("成交金額").fillna(0) / 1e8
成交量 = data.get("成交股數").fillna(0) / 1000
周均量 = 成交量.rolling(5).mean()
月均量 = 成交量.rolling(20).mean()
# 成交值
當日成交值排行 = 成交值.rank(axis=1, ascending=False)

# 均線指標
ma60 = 收盤價.rolling(60, min_periods=10).mean().round(2)
ma40 = 收盤價.rolling(40, min_periods=10).mean().round(2)
ma30 = 收盤價.rolling(30, min_periods=10).mean().round(2)
ma20 = 收盤價.rolling(20, min_periods=10).mean().round(2)
ma10 = 收盤價.rolling(10).mean().round(2)
ma5 = 收盤價.rolling(5).mean().round(2)

# 布林通道
upper, middle, lower = data.talib('BBANDS', timeperiod=20, nbdevup=2, nbdevdn=2)

# 均線趨勢
def rising_curve(sma, n):
    return (sma.shift(n) + sma) / 2 >= sma.shift(round(0.5 * (n+1)))

rising_20 = (
    rising_curve(ma20, 5) &
    rising_curve(ma20, 10) & 
    rising_curve(ma20, 15) &
    (收盤價 >= ma20)
)
rising_40 = (
    rising_curve(ma40, 5) &
    rising_curve(ma40, 10) & 
    rising_curve(ma40, 15) &
    (收盤價 > ma40)
)
rising_60 = (
    rising_curve(ma60, 5) &
    rising_curve(ma60, 10) & 
    rising_curve(ma60, 15) &
    rising_curve(ma60, 20) &
    (收盤價 > ma60)
)

# === 籌碼面指標資料 ===
外資買賣超 = data.get("外資買賣超股數").fillna(0) / 1000
投信買賣超 = data.get("投信買賣超股數").fillna(0) / 1000
法人買賣超 = 外資買賣超 + 投信買賣超

# 投信成交值排行(賣超排行用-號)
投信成交值 = 投信買賣超 * 收盤價
投信買金額排行 = 投信成交值.rank(axis=1, ascending=False)
投信賣金額排行 = -1 * 投信成交值.rank(axis=1, ascending=True)
df投信買 = (投信買賣超 > 0)
df投信買 = df投信買.replace({True:np.nan}).fillna(投信買金額排行)
df投信買 = df投信買.replace({False:np.nan})
df投信賣 = (投信買賣超 < 0)
df投信賣 = df投信賣.replace({True:np.nan}).fillna(投信賣金額排行)
df投信賣 = df投信賣.replace({False:np.nan})
投信買賣金額排行 = df投信買.fillna(df投信賣).fillna(0).astype(int)

外資成交值 = 外資買賣超 * 收盤價
外資買金額排行 = 外資成交值.rank(axis=1, ascending=False)
外資賣金額排行 = -1 * 外資成交值.rank(axis=1, ascending=True)
df外資買 = (外資買賣超 > 0)
df外資買 = df外資買.replace({True:np.nan}).fillna(外資買金額排行)
df外資買 = df外資買.replace({False:np.nan})
df外資賣 = (外資買賣超 < 0)
df外資賣 = df外資賣.replace({True:np.nan}).fillna(外資賣金額排行)
df外資賣 = df外資賣.replace({False:np.nan})
外資買賣金額排行 = df外資買.fillna(df外資賣).fillna(0).astype(int)

# 投信、外資連續買超天數
df_count = pd.DataFrame(index=收盤價.index, columns=收盤價.columns, data=1)
df_count = df_count.cumsum(axis=0)
投信多久沒賣 = ((投信買賣超 <= 0).rolling(2).sum() == 2)
投信多久沒賣 = 投信多久沒賣.replace({True:np.nan}).fillna(df_count)
投信多久沒賣 = 投信多久沒賣.replace({False:np.nan}).fillna(method='ffill')
投信多久沒賣 = df_count - 投信多久沒賣

外資多久沒賣 = ((外資買賣超 <= 0).rolling(2).sum() == 2)
外資多久沒賣 = 外資多久沒賣.replace({True:np.nan}).fillna(df_count)
外資多久沒賣 = 外資多久沒賣.replace({False:np.nan}).fillna(method='ffill')
外資多久沒賣 = df_count - 外資多久沒賣

# === 基本面指標資料 (營收) ===
MOM = data.get('上月比較增減(%)')
YOY = data.get('去年同月增減(%)')
累計YOY = data.get('前期比較增減(%)')
去年當月營收 = data.get('去年當月營收')
當月營收 = data.get('當月營收')
近3月營收年增率 = round((當月營收.rolling(3).sum() / 去年當月營收.rolling(3).sum() - 1) * 100, 2)
近6月營收年增率 = round((當月營收.rolling(6).sum() / 去年當月營收.rolling(6).sum() - 1) * 100, 2)
近12月營收年增率 = round((當月營收.rolling(12).sum() / 去年當月營收.rolling(12).sum() - 1) * 100, 2)

# === 基本面指標資料 (財報) ===
# 成長率指標
def 成長率(s):
    return (s / s.shift(4) - 1) * 100

股本 = data.get("股本合計")
股本 = 股本.fillna(method='ffill')
EPS = data.get("基本每股盈餘合計")
本益比 = data.get("本益比")
T3100營業收入淨額 = data.get("營業收入合計")
T3295營業毛利 = data.get('營業毛利（毛損）').fillna(data.get('營業毛利（毛損）淨額'))
T3395營業利益 = data.get('營業利益（損失）').fillna(data.get('營業利益'))
# 注意 income_sheet 是單季，balance_sheet是累計
T3900繼續營業部門稅前純益 = data.get('繼續營業單位稅前淨利（淨損）', table='income_sheet')
T3970經常稅後淨利 = data.get('本期淨利（淨損）').fillna(data.get('本期稅後淨利（淨損）'))
T3700營業外收入及支出 = data.get("營業外收入及支出合計").fillna(data.get("營業外損益合計"))
T3300營業費用 = data.get("營業費用合計")
T0170存貨 = data.get("存貨合計").fillna(data.get("存貨"))

R105_營業毛利率 = T3295營業毛利 / T3100營業收入淨額 *100
R108_稅後淨利率 = T3970經常稅後淨利 / T3100營業收入淨額 *100

R402_營業毛利成長率 = 成長率(T3295營業毛利)
R405_稅後淨利成長率 = 成長率(T3970經常稅後淨利)


#--------------------------------------------------------------------
df_上市 = pd.read_csv('上市產業分類.csv', encoding='utf8')
df_上櫃 = pd.read_csv('上櫃產業分類.csv', encoding='utf8')
df_上市 = df_上市[['公司代號', '公司簡稱', '產業類別']].rename(columns={'公司代號': 'stock_id', '公司簡稱':'name'})
df_上市 = df_上市.astype(str).set_index('stock_id').sort_index()
df_上櫃 = df_上櫃[['公司代號', '公司簡稱', '產業類別']].rename(columns={'公司代號': 'stock_id', '公司簡稱':'name'})
df_上櫃 = df_上櫃.astype(str).set_index('stock_id').sort_index()
df產業類別 = pd.concat([df_上市, df_上櫃]).sort_index()
df細產業 = pd.read_pickle(os.path.join('MoneyDJ', 'stock_MoneyDJ.pkl'))

df_category_pe = pd.read_csv('產業本益比表.csv', index_col=0)

def create_money_dj():
    df細產業 = pd.DataFrame()
    group_path = 'MoneyDJ'
    for stock_group in os.listdir(group_path):

        if stock_group[-4:] != '.csv':
            continue

        try:
            stocks = pd.read_csv(os.path.join(group_path, stock_group), encoding='utf8')
        except:
            print(stock_group)
            continue

        df細產業 = df細產業.append(stocks, ignore_index=True)

    df細產業 = df細產業.rename(columns={'代號': 'stock_id', '名稱':'name'})
    df細產業 = df細產業.astype(str).set_index('stock_id').sort_index()
    df細產業.to_pickle(os.path.join(group_path, 'stock_MoneyDJ.pkl'))
    
def pick_industry_stocks(indeustry_level='細產業', indeustry_name=''):
    return df細產業[df細產業[indeustry_level] == indeustry_name]

def get_industry_name(stock_id, indeustry_level='細產業'):
    
    if stock_id not in df細產業.index:
        return "無產業資訊"
    
    rtn = df細產業.loc[stock_id, indeustry_level]
    if isinstance(rtn, str):
        return rtn
    else:
        return rtn.drop_duplicates().to_list()
    
def get_stock_name(stock_id):
    
    if stock_id not in df產業類別.index:
        return f"{stock_id} 不在清單"
    
    stock_name = df產業類別.loc[stock_id]['name']
    return stock_name if isinstance(stock_name, str) else stock_name.iloc[0]

def get_category(stock_id):
    
    if stock_id not in df產業類別.index:
        return "無產業資訊"
    return df產業類別.loc[stock_id, '產業類別']

def list_same_category(category, detail):
    if detail == "":
        return df產業類別[df產業類別['產業類別'] == category].index
    else:
        return df產業類別[df產業類別['產業類別'] == category].index & df細產業[df細產業['細產業'] == detail].index
    
def get_category_pe(sid, method='min'):
    
    try:
        if sid not in df細產業.index:
            print(f'Warning: {sid} not in 細產業')
            return 15

        category = df細產業.loc[sid, '細產業']
        if method =='min':
            if len(df_category_pe.loc[category]) > 1:
                return df_category_pe.loc[category].min()[0]
            else:
                return df_category_pe.loc[category].min()
        elif method in df_category_pe.index:
            return df_category_pe.loc[method][0]
        else:
            return df_category_pe.loc[category]
    except:
        print(f'異常: {sid}')

In [ ]:
class TwStockTreeMap:

    def __init__(self, close, basic_info, start=None, end=None):
        self.start = start
        self.end = end
        self.close = close
        self.basic_info = basic_info

    # dataframe filter by selected date
    def df_date_filter(self, df, start=None, end=None):
        if start:
            df = df[df.index >= start]
        if end:
            df = df[df.index <= end]
        return df

    # map stock_name from basic info(stock_id +name)
    def map_stock_name(self, basic_info, s):
        target = basic_info[basic_info['stock_id'].str.find(s) > -1]
        if len(target) > 0:
            s = target['stock_id'].values[0]
        return s

    def create_data(self):
        close_data = self.df_date_filter(self.close, self.start, end=self.end)
        trans_value = self.df_date_filter(成交值, self.start, end=self.end)
        return_ratio = (close_data.iloc[-1] / close_data.iloc[0]).dropna().replace(np.inf, 0)
        return_ratio = round((return_ratio - 1) * 100, 2)
        return_ratio = pd.concat([return_ratio, close_data.iloc[-1], trans_value.iloc[-1]], axis=1, sort=False).dropna()
        return_ratio = return_ratio.reset_index()
        return_ratio.columns = ['stock_id', 'return_ratio', 'close', 'vol']
        return_ratio['stock_id'] = return_ratio['stock_id'].apply(lambda s: self.map_stock_name(basic_info, s))
        return_ratio = return_ratio.merge(self.basic_info[['stock_id', '產業類別', '市場別', '實收資本額(元)']], how='left',
                                          on='stock_id')
        return_ratio = return_ratio.rename(columns={'產業類別': 'category', '市場別': 'market', '實收資本額(元)': 'base'})
        return_ratio['market_value'] = round(return_ratio['base'] / 10 * return_ratio['close'], 2)
        return_ratio = return_ratio.dropna(thresh=5)
        return_ratio['country'] = 'TW-Stock'
        return_ratio['return_ratio_text_info']=return_ratio['return_ratio'].astype(str).apply(lambda s: '+' + s if '-' not in s else s) + '%'
        return return_ratio

    def create_fig(self, relative_market_strength=False):
        df = self.create_data()
        if relative_market_strength is True:
            color_continuous_midpoint = np.average(df['return_ratio'], weights=df['base'])
        else:
            color_continuous_midpoint = 0
        fig = px.treemap(df, 
                         path=['country', 'market', 'category', 'stock_id'], 
                         values='market_value',
                         color='return_ratio',
                         color_continuous_scale='Tealrose',
                         color_continuous_midpoint=color_continuous_midpoint,
                         custom_data=['return_ratio_text_info','close'],
                         title=f'TW-Stock Market TreeMap({self.start}~{self.end})',
                         width=1200, 
                         height=900)
        
        fig.update_traces(textposition='middle center', 
                          textfont_size=20,
                          texttemplate= "%{label}<br>%{customdata[0]}<br>%{customdata[1]}"
                          )
        return fig

#@title 台股漲跌與市值板塊圖
# start= '2021-05-31' #@param {type:"date"}
# end = '2021-06-01' #@param {type:"date"}
start = 收盤價.index[-2].date().strftime("%Y-%m-%d")
end = 收盤價.index[-1].date().strftime("%Y-%m-%d")
relative_market_strength = "False" #@param ["False", "True"] {type:"raw"}

stock_ids = (收盤價.columns & df產業類別.index & 股本.columns).tolist()
stock_id = ['{} {}'.format(sid, df產業類別.loc[sid, 'name']) for sid in stock_ids]
產業類別 = [df產業類別.loc[sid, '產業類別'] for sid in stock_ids]
市場別 = ['twii' if sid in df_上市.index else 'otc' for sid in stock_ids]

basic_info = pd.DataFrame(index=stock_ids, data={'stock_id': stock_id, '產業類別':產業類別, '市場別':市場別, '實收資本額(元)':股本[stock_ids].iloc[-1]})

TwStockTreeMap(收盤價[stock_ids],basic_info,start,end).create_fig(relative_market_strength)

In [ ]:
sids = df產業類別.index.drop_duplicates() & 收盤價.columns
# 計算所有類股周月乖離率
漲跌幅 = round(percentage(收盤價[sids], 收盤價[sids].shift(1)), 2)
周乖離 = round(percentage(收盤價[sids], ma5[sids]), 2)
月乖離 = round(percentage(收盤價[sids], ma20[sids]), 2)
# 計算強/弱勢股票數量
強勢股 = ((周乖離 >= 0) & (月乖離 >= 0.5)).fillna(False).apply(pd.value_counts, axis=1)
弱勢股 = ((周乖離 < 0) & (月乖離 < -0.5)).fillna(False).apply(pd.value_counts, axis=1)

漲幅1 = round(percentage(收盤價, 收盤價.shift(1)), 2)
漲幅5 = round(percentage(收盤價, 收盤價.shift(5)), 2)
漲幅20 = round(percentage(收盤價, 收盤價.shift(20)), 2)

連續進榜 = 當日成交值排行 > 50
連續進榜 = 連續進榜.replace({True:np.nan}).fillna(df_count)
連續進榜 = 連續進榜.replace({False:np.nan}).fillna(method='ffill')
連續進榜 = df_count - 連續進榜

In [ ]:
import datetime
import pandas as pd
import os
import matplotlib.pyplot as plt
from finlab.data import Data
from finlab.plot_candles import plot_candles

# sd = datetime.date(2021, 2, 1)
sd = 收盤價.index[-40].date()
ed = 收盤價.index[-1].date()
# ed = datetime.date(2021, 5, 13)

強勢股數量 = 強勢股[True].loc[ed]
強勢股增加 = 強勢股[True].loc[ed] - 強勢股[True].shift(1).loc[ed]
弱勢股數量 = 弱勢股[True].loc[ed]
弱勢股增加 = 弱勢股[True].loc[ed] - 弱勢股[True].shift(1).loc[ed]

plt.rcParams["figure.figsize"] = (20,8)
pricing = pd.DataFrame({'open': 指數開盤價.loc[sd:ed, '發行量加權股價指數'], 
                        'close': 指數收盤價.loc[sd:ed, '發行量加權股價指數'], 
                        'high': 指數最高價.loc[sd:ed, '發行量加權股價指數'], 
                        'low': 指數最低價.loc[sd:ed, '發行量加權股價指數'],
                        'volume':指數成交值.loc[sd:ed, '發行量加權股價指數']})

plt.rcParams["figure.figsize"] = (20,10)
plot_candles(start_time=sd, end_time=ed, pricing=pricing,volume_bars=True, 
             overlays=[指數收盤價ma5.loc[sd:ed, '發行量加權股價指數'], 指數收盤價ma10.loc[sd:ed, '發行量加權股價指數'],
                       指數收盤價ma20.loc[sd:ed, '發行量加權股價指數'], 指數收盤價ma60.loc[sd:ed, '發行量加權股價指數']], 
             technicals=[pd.concat([強勢股[True].rolling(1).mean(), 弱勢股[True].rolling(1).mean()], axis=1)])

print(f'{ed} 強勢股: {強勢股數量:.0f} ({強勢股增加:.0f}), 弱勢股: {弱勢股數量:.0f} ({弱勢股增加:.0f})')
plt.show()


In [ ]:
def show_trigger(condi_name, condi):
    
    trigger = condi.loc[date, :]
    satify = trigger[trigger].index.tolist()
    print(f'{condi_name}: {satify}')

date = 收盤價.index[-1].date()
# date = datetime.date(2021, 6, 25)

突破月線 = (指數收盤價 >= 指數收盤價ma20) & (指數收盤價.shift(1) < 指數收盤價ma20.shift(1))
突破季線 = (指數收盤價 >= 指數收盤價ma60) & (指數收盤價.shift(1) < 指數收盤價ma60.shift(1)) & (指數收盤價 >= 指數收盤價ma20)
創60新高 = (指數最高價 >= 指數最高價.shift(1).rolling(60).max()) & (指數最高價.shift(1) < 指數最高價.shift(1).rolling(60).max())
出量 = (指數成交值 > 指數成交值ma5) & (指數成交值.shift(1) < 指數成交值ma5.shift(1))
漲幅 = percentage(指數收盤價, 指數收盤價.shift(1)) > 3
大紅K = percentage(指數收盤價, 指數開盤價.shift(1)) > 3

print(date)
show_trigger('突破月線', 突破月線)
show_trigger('突破月季線', 突破季線)
show_trigger('創60新高', 創60新高)
show_trigger('出量', 出量)
show_trigger('漲幅3%', 漲幅)
show_trigger('紅棒3%', 大紅K)


In [ ]:
創60新高[創60新高['電子零組件類指數']]['電子零組件類指數'].tail(10)

In [ ]:
def set_percentage_color(s):

    return ['color: red' if v > 0 else 'color: green' for v in s]
    
強勢類股condi1 = 指數收盤價 > 0
twii_index = 指數收盤價ma5.drop(columns=['電子類指數', '發行量加權股價指數']).columns
show_index = 強勢類股condi1.loc[date, 強勢類股condi1.loc[date]].index & twii_index

指數漲幅 = round(percentage(指數收盤價[twii_index], 指數收盤價[twii_index].shift(1)), 2)
指數K棒 = round(percentage(指數收盤價[twii_index], 指數開盤價[twii_index]), 2)
指數均量比 = round(percentage(指數成交值[twii_index], 指數成交值ma5[twii_index].shift(1)), 2)
指數月乖離 = round(percentage(指數收盤價[twii_index], 指數收盤價ma20[twii_index]), 2)
指數成交值排名 = 指數成交值[twii_index].rank(ascending=False, axis=1).astype(int)
index_info = pd.DataFrame(index=指數成交值[twii_index].columns, data={'值':指數成交值排名.loc[date].tolist()})
index_info['成交值'] = 指數成交值[twii_index].loc[date]
index_info['量'] = 指數均量比.loc[date].rank(ascending=False).astype(int)
index_info['量能'] = 指數均量比.loc[date]
index_info['漲'] = 指數漲幅.loc[date].rank(ascending=False).astype(int)
index_info['漲幅'] = 指數漲幅.loc[date]
index_info['K'] = 指數K棒.loc[date].rank(ascending=False).astype(int)
index_info['K棒'] = 指數K棒.loc[date]
index_info['離'] = 指數月乖離.loc[date].rank(ascending=False).astype(int)
index_info['月乖離'] = 指數月乖離.loc[date]

print(date)
print('乖離排行')
index_info = index_info.loc[show_index].sort_values('月乖離', ascending=False)
display(index_info.head(8).style.apply(set_percentage_color, subset=['漲幅', 'K棒', '量能', '月乖離',]))
print('漲幅排行')
index_info = index_info.loc[show_index].sort_values('漲幅', ascending=False)
display(index_info.head(5).style.apply(set_percentage_color, subset=['漲幅', 'K棒', '量能', '月乖離',]))
print('量能排行')
index_info = index_info.loc[show_index].sort_values('量能', ascending=False)
display(index_info.head(5).style.apply(set_percentage_color, subset=['漲幅', 'K棒', '量能', '月乖離',]))
print('成交排行')
index_info = index_info.loc[show_index].sort_values('成交值', ascending=False)
display(index_info.head(8).style.apply(set_percentage_color, subset=['漲幅', 'K棒', '量能', '月乖離',]))

In [ ]:
指數成交值排名['電子零組件類指數'].tail(60).plot()
# 指數月乖離['半導體類指數'].tail(90).plot()
# 指數月乖離['航運類指數'].tail(90).plot()
# 指數月乖離['貿易百貨類指數'].tail(90).plot()

In [ ]:
sd = 收盤價.index[-30].date()
ed = 收盤價.index[-1].date()

漲幅 = round(percentage(指數收盤價, 指數收盤價.shift(1)), 2)
K棒 = round(percentage(指數收盤價, 指數開盤價), 2)
for c_index in index_info.index:
    
    plt.rcParams["figure.figsize"] = (20,6)
    pricing = pd.DataFrame({'open': 指數開盤價.loc[sd:ed, c_index], 
                        'close': 指數收盤價.loc[sd:ed, c_index], 
                        'high': 指數最高價.loc[sd:ed, c_index], 
                        'low': 指數最低價.loc[sd:ed, c_index],
                        'volume':指數成交值.loc[sd:ed, c_index]})
    
    
    hint = f'{ed}, {c_index}, 漲幅:{漲幅.loc[ed, c_index]}%, K棒:{K棒.loc[ed, c_index]}%'
    
    plot_candles(start_time=sd, end_time=ed, pricing=pricing,volume_bars=True, 
                 title=hint,
                 overlays=[指數收盤價ma5.loc[sd:ed, c_index], 指數收盤價ma10.loc[sd:ed, c_index], 
                           指數收盤價ma20.loc[sd:ed, c_index], 指數收盤價ma60.loc[sd:ed, c_index]], )
    plt.show()
    

In [ ]:
def set_pricing_color(s):

    if s.name in ['投信', '外資']:
        return ['' if v == 0 else ('color: blue' if v > 0 else 'color: darkorange')  for v in s]
    elif s.name in ['漲幅1','漲幅5','漲幅20']:
        return ['' if v == 0 else ('color: red' if v > 0 else 'color: green')  for v in s]
    elif s.name in ['新進']:
        return ['background-color: yellow' if v == 'o' else '' for v in s]
    else:
        return ['' for v in s]

    
# date = datetime.date(2021, 6, 25)
date = 收盤價.index[-1].date()

print(date)
過去累計天數 = 5
排行數 = 50
過去進榜 = (當日成交值排行.shift(1) <= 排行數).rolling(過去累計天數).sum() > 0
過去熱門股 = 過去進榜[過去進榜].loc[:date].iloc[-1].dropna().index
過去熱門股 = set(過去熱門股) & set(df產業類別.index)
當日熱門股 = 當日成交值排行.loc[:date].iloc[-1].sort_values().iloc[:排行數].index
當日熱門股 = set(當日熱門股) & set(df產業類別.index)
新進熱門股 = set(當日熱門股) - set(過去熱門股)

顯示對象 = 當日熱門股 & set(df產業類別.index.tolist())
val = valuation_stocks(date, date, 顯示對象)

df_當日 = df產業類別.loc[顯示對象, ['name']].drop_duplicates(['name'])
df_當日['收盤'] = 收盤價.loc[date, 顯示對象]
df_當日['市場'] = ['twii' if sid in df_上市.index else 'otc' for sid in 顯示對象]
df_當日['產業別'] = [df產業類別.loc[sid, '產業類別'] for sid in 顯示對象]
df_當日['新進'] =['x'] * len(df_當日)
df_當日.loc[新進熱門股, '新進'] = ['o'] * len(新進熱門股)
df_當日['排序'] = 當日成交值排行.loc[date, 顯示對象].astype('int')
df_當日['投信'] = 投信買賣金額排行.loc[date, 顯示對象].astype('int')
df_當日['外資'] = 外資買賣金額排行.loc[date, 顯示對象].astype('int')
df_當日['漲幅1'] = 漲幅1.loc[date, 顯示對象]
df_當日['漲幅5'] = 漲幅5.loc[date, 顯示對象]
df_當日['漲幅20'] = 漲幅20.loc[date, 顯示對象]
df_當日['次數'] = 連續進榜.loc[date, 顯示對象].astype('int')
df_當日['投信連'] = 投信多久沒賣.loc[date, 顯示對象].astype('int')
df_當日['外資連'] = 外資多久沒賣.loc[date, 顯示對象].astype('int')
df_當日['pe'] = [get_category_pe(sid) for sid in 顯示對象]
df_當日['估值'] = val.astype('int').tolist()
df_當日['細產業'] = [get_industry_name(sid,'產業別') for sid in 顯示對象]
df_當日 = df_當日.sort_values('排序')

pd.set_option('max_colwidth', 100)

熱門法人同買 = df_當日[(df_當日['投信'] > 0) & (df_當日['外資'] > 0)]
display(df_當日[df_當日['新進'] == 'o'].style.apply(set_pricing_color))
display(熱門法人同買.style.apply(set_pricing_color))
display(df_當日.style.apply(set_pricing_color).format({'漲幅1': "{:+.2f}", '漲幅5': "{:+.2f}", '漲幅20': "{:+.2f}"}))
#---------------------------------------------
熱門產業counts = df產業類別.loc[當日熱門股, '產業類別'].value_counts()
新進產業counts = df產業類別.loc[新進熱門股, '產業類別'].value_counts()

list_細產業 = []
for c in 熱門產業counts.index:
    細產業counts = df細產業.loc[df_當日[df_當日['產業別'] == c].index, '細產業'].value_counts().head(10)
    tmp = ['{}({})'.format(i, 細產業counts[i]) for i in 細產業counts.index]
    list_細產業.append(tmp)

df熱門 = pd.DataFrame(index=熱門產業counts.index, data={'今日': 熱門產業counts.values})
df新進 = pd.DataFrame(index=新進產業counts.index, data={'新進': 新進產業counts.values})
df熱門 = pd.concat([df熱門, df新進], axis=1, sort=False)
df熱門['新進'] = df熱門['新進'].fillna(0).astype('int')
df熱門['細產業統計'] = list_細產業
pd.set_option('max_colwidth', 200)
display(df熱門)

In [ ]:
main_category = '半導體業'
sub_category = '類比IC'
sids = list_same_category(main_category, sub_category) & 當日成交值排行.columns

date = datetime.date(2021, 6, 29)

df同類股 = df產業類別.loc[sids, ['name']]
市值 = 股本.loc[:date, sids].iloc[-1] * 收盤價.loc[:date, sids].iloc[-1]
存貨市值比 = round(10 * T0170存貨.loc[:date, sids].iloc[-1] / 市值, 2)

df同類股['市值'] = 市值.rank(ascending=False).astype('int')
df同類股['成交排行'] = 當日成交值排行.loc[date, sids].astype('int')
df同類股['投信'] = 投信買賣金額排行.loc[date, sids].astype('int')
df同類股['外資'] = 外資買賣金額排行.loc[date, sids].astype('int')
df同類股['營收(億)'] = round(當月營收.loc[:date, sids].iloc[-1] / 10e4, 2)
df同類股['月均量(千)'] = round(月均量.loc[date, sids]/1000, 2)
df同類股['YOY'] = 近3月營收年增率.loc[:date, sids].iloc[-1]
df同類股['存貨市值比'] = 存貨市值比
df同類股['毛利'] = round(R105_營業毛利率.loc[:date, sids].iloc[-1], 2)
df同類股['毛利差'] = round(R105_營業毛利率.loc[:date, sids].iloc[-1] - R105_營業毛利率.loc[:date, sids].iloc[-3], 2)
df同類股['細產業'] =[get_industry_name(sid) for sid in sids]
df同類股.sort_values('成交排行', ascending=True, inplace=True)
print(f'{date}: {main_category}-{sub_category} ({len(df同類股)})')
df同類股 = df同類股.head(10)
display(df同類股)

def 法人連續買超(選股條件 = []):
    
    n=3
    近日投信有連買 = (投信多久沒賣 >= n).rolling(3).sum() > 0
    近日外資有連買 = (外資多久沒賣 >= n).rolling(3).sum() > 0
    選股條件.append(近日投信有連買 | 近日外資有連買)
    
    選股條件.append(YOY > 50)
    選股條件.append((近3月營收年增率 > 近12月營收年增率) & (近6月營收年增率 > 近12月營收年增率))
    選股條件.append(近6月營收年增率 > 近6月營收年增率.shift(1))
    選股條件.append(R105_營業毛利率 > R105_營業毛利率.shift(2))
    選股條件.append(R105_營業毛利率 > R105_營業毛利率.shift(3))
    
    return 選股條件

my_strategy = 法人連續買超([])
pick_list = pick_stocks(date, sids, my_strategy, 1)

df法人買超 = df產業類別.loc[pick_list, ['name']]
df法人買超['投信'] = 投信買賣金額排行.loc[date, pick_list].astype('int')
df法人買超['外資'] = 外資買賣金額排行.loc[date, pick_list].astype('int')
df法人買超['營收(億)'] = round(當月營收.loc[:date, pick_list].iloc[-1] / 10e4, 2)
df法人買超['月均量(千)'] = round(月均量.loc[date, pick_list]/1000, 2)
df法人買超['YOY'] = 近3月營收年增率.loc[:date, pick_list].iloc[-1]
df法人買超['毛利差'] = round(R105_營業毛利率.loc[:date, pick_list].iloc[-1] - R105_營業毛利率.loc[:date, pick_list].iloc[-3], 2)
df法人買超['細產業'] =[get_industry_name(sid) for sid in pick_list]
df法人買超

In [ ]:
"""
1. 成交值排名
1. YOY 排名
2. 毛利率排名
3. 營業利益率排名
4. 投信3日內買超排名
5. 投信10日內買超排名
6. 投信連續買超天數排名
6. 外資3日內買超排名
7. 外資10日內買超排名
8. 外資連續買超天數排名
9. 
"""

n = 3
# date = datetime.date(2021,6,11)

成交值前5 = 成交值.loc[date, sids].sort_values(ascending=False)
YOY前5 = YOY.loc[:date, sids].iloc[-1].sort_values(ascending=False)
MOM前5 = MOM.loc[:date, sids].iloc[-1].sort_values(ascending=False)

毛利差 = R105_營業毛利率 - R105_營業毛利率.shift(1)
毛利差前5 = 毛利差.loc[:date, sids].iloc[-1].sort_values(ascending=False)
淨利差 = R108_稅後淨利率 - R108_稅後淨利率.shift(1)
淨利差前5 = 淨利差.loc[:date, sids].iloc[-1].sort_values(ascending=False)

投信3日買超前5 = 投信買賣超.rolling(3).sum().loc[date, sids].sort_values(ascending=False)
投信10日買超前5 = 投信買賣超.rolling(10).sum().loc[date, sids].sort_values(ascending=False)
投信連買前5 = 投信多久沒賣.loc[date, sids].sort_values(ascending=False)
外資3日買超前5 = 外資買賣超.rolling(3).sum().loc[date, sids].sort_values(ascending=False)
外資10日買超前5 = 外資買賣超.rolling(10).sum().loc[date, sids].sort_values(ascending=False)
外資連買前5 = 外資多久沒賣.loc[date, sids].sort_values(ascending=False)

前5列表 = 成交值前5.index[0:n] | YOY前5.index[0:n] | MOM前5.index[0:n] | \
    毛利差前5.index[0:n] | 淨利差前5.index[0:n] | \
    投信3日買超前5.index[0:n] | 投信10日買超前5.index[0:n] | 投信連買前5.index[0:n] | \
    外資3日買超前5.index[0:n] | 外資10日買超前5.index[0:n] | 外資連買前5.index[0:n]

df類股各項排名 = df產業類別.loc[前5列表, ['name']]
df類股各項排名['成交'] = 成交值前5.loc[前5列表].rank(ascending=False).astype('int')
df類股各項排名['YOY'] = YOY前5.loc[前5列表].rank(ascending=False).astype('int')
df類股各項排名['MOM'] = MOM前5.loc[前5列表].rank(ascending=False).astype('int')
df類股各項排名['毛利差'] = 毛利差前5.loc[前5列表].rank(ascending=False).astype('int')
df類股各項排名['淨利差'] = 淨利差前5.loc[前5列表].rank(ascending=False).astype('int')

df類股各項排名['投信3'] = 投信3日買超前5.loc[前5列表].rank(ascending=False).astype('int')
df類股各項排名['投信10'] = 投信10日買超前5.loc[前5列表].rank(ascending=False).astype('int')
df類股各項排名['投信連'] = 投信連買前5.loc[前5列表].rank(ascending=False).astype('int')
df類股各項排名['外資3'] = 外資3日買超前5.loc[前5列表].rank(ascending=False).astype('int')
df類股各項排名['外資10'] = 外資10日買超前5.loc[前5列表].rank(ascending=False).astype('int')
df類股各項排名['外資連'] = 外資連買前5.loc[前5列表].rank(ascending=False).astype('int')
df類股各項排名['漲幅'] = round(percentage(收盤價.loc[date:, 前5列表].max(), 收盤價.loc[date, 前5列表]), 2)

df類股各項排名.sort_values('成交', ascending=True, inplace=True)
df類股各項排名

In [ ]:
end_num = 0

# 繪製K線圖
# 熱門法人同買
for sid in 熱門法人同買.index:
    
    show_len = end_num if len(收盤價.loc[date:, sid]) > end_num else -1
    
    pricing = pd.DataFrame({'open': 開盤價[sid], 
                            'close': 收盤價[sid], 
                            'high': 最高價[sid], 
                            'low': 最低價[sid], 
                            'volume': 成交量[sid]})
    hint = '{}, {}, {} {}'.format(get_category(sid), date.strftime('%Y-%m-%d'), sid, get_stock_name(sid))
    
    plt.rcParams["figure.figsize"] = (20,8)
    plot_candles(start_time=收盤價.loc[:date].index[-59], 
                 end_time=收盤價.loc[date:].index[show_len], 
                 pricing=pricing, volume_bars=True,
                 title=hint, 
                 overlays=[ma20[sid], ma60[sid], ma10[sid], ma5[sid]])
    
    當天漲幅 = round(percentage(收盤價.loc[:date, sid][-1], 收盤價.loc[:date, sid][-2]), 2)
    限制價位 = round(收盤價.loc[:date, sid][-1] * 1.02, 2)
    print('{}, {}: {}, 當天漲幅: {}, 限制價位: {}'.format(date, 
                                    get_stock_name(sid), get_industry_name(sid), 當天漲幅, 限制價位))
    轉強日, 轉強天數, 轉強漲幅 = get_trend_up_date(date, sid)    
    print(f'{轉強日} ({轉強天數}), 轉強後漲幅: {轉強漲幅}%')
        
    預估周線價格 = round((收盤價.loc[:date, sid].rolling(4).sum().iloc[-1] + 收盤價.loc[date, sid]) / 5, 2)
    預估月線價格 = round((收盤價.loc[:date, sid].rolling(19).sum().iloc[-1] + 收盤價.loc[date, sid]) / 20, 2)
    
    預估周線乖離 = round(percentage(收盤價.loc[date, sid], 預估周線價格), 2)
    預估月線乖離 = round(percentage(收盤價.loc[date, sid], 預估月線價格), 2)
    print(f'周線乖離: {預估周線價格}({預估周線乖離}%), 月線乖離: {預估月線價格}({預估月線乖離}%)')
    print('成交值: {}'.format(當日成交值排行.loc[:date, sid].tail(5).astype(int).tolist()))
    print('投信買: {}'.format(投信買賣金額排行.loc[:date, sid].tail(5).tolist()))
    print('外資買: {}'.format(外資買賣金額排行.loc[:date, sid].tail(5).tolist()))

    plt.show()
    
    draw_finance(date, sid)

In [ ]:
sid = '5269'
s_name = df產業類別.loc[sid, 'name']
date = datetime.date(2020, 5, 15)
inventory = 1
sell_stock = 0
sell_date = date
ready_stop = False
trade_info = {}

買進價 = 開盤價.loc[date:, sid][1]
開盤漲幅 = percentage(開盤價.loc[date:, sid][1], 收盤價.loc[date, sid])
最低漲幅 = percentage(最低價.loc[date:, sid][1], 收盤價.loc[date, sid])
print(f'{date}, {sid} {s_name}')
print(f'當日漲幅: {漲幅1.loc[date, sid]}%, 月乖離: {月乖離.loc[date, sid]}%')
print(f'買進價: {買進價}, 開盤漲幅: {開盤漲幅:+.2f}%, 最低漲幅: {最低漲幅:+.2f}%')

停損價 = ma20.loc[date, sid]
最高獲利 = 0
for d in 收盤價.loc[date:, sid].index[1:]:
    
    current_price = 收盤價.loc[d, sid]
    周線乖離 = percentage(current_price, ma10.loc[d, sid])
    月線乖離 = percentage(current_price, ma20.loc[d, sid])
    季線乖離 = percentage(current_price, ma60.loc[d, sid])
    目前最高 = 最高價.loc[date:d, sid].max()
    
    最大回檔 = percentage(current_price, 目前最高)
    獲利 = percentage(current_price, 買進價)
    
    主要均線 = 月線乖離
    if (主要均線 < -0.35) & (ready_stop == False):
        ready_stop = True
        if ((月線乖離 > 0) and (月線乖離 < 3.5)) or ((季線乖離 > 0) and (季線乖離 < 3.5)):
            print(f'{d:%Y-%m-%d} 改月季支撐, 月乖離: {月線乖離:+.2f}%, 季乖離: {季線乖離:+.2f}%')
        else:
            停損價 = 收盤價.loc[d, sid]
            print(f'{d:%Y-%m-%d} 破線支撐, 更新停損價: {停損價}, 月乖離: {月線乖離:+.2f}%, 季乖離: {季線乖離:+.2f}%')
    
    if (ready_stop) & (主要均線 > 0):
        ready_stop = False
        
    if (最高獲利 > 20) and (最大回檔 < -8) and (inventory == 1):
        inventory = 0.5
        print(f'{d:%Y-%m-%d} 回檔: {最大回檔:+.2f}%, 賣出一半, 獲利: {獲利:+.2f}%')
        
        
    if (percentage(current_price, 停損價) < -0.2):
        inventory = inventory - 1
        sell_date = d
        持有天數 = len(收盤價.loc[date:d, sid]) + 1
        獲利 = percentage(current_price, 買進價)
        期間最高價 = 最高價.loc[date:d, sid].max()
        獲利 = percentage(current_price, 買進價)
        
        max_profit_date = 最高價.loc[date:d, sid].idxmax().date()
        max_profit_days = 收盤價.loc[date:max_profit_date, sid].size - 1
        max_profit = round(percentage(最高價.loc[max_profit_date, sid], 買進價), 2)
        print(f'最高獲利: {max_profit_date:%Y-%m-%d} ({max_profit_days}), 獲利: {max_profit:+.2f}%')
        print(f'賣出日: {d:%Y-%m-%d} ({持有天數}), 獲利: {獲利:+.2f}%')
        
    if inventory <= 0:
        break
        
    
    最高獲利 = percentage(目前最高, 買進價)
    if d == 收盤價.loc[date:, sid].index[-1]:
        print(f'仍持有中: 獲利: {獲利:+.2f}%')
    

In [ ]:
def 法人連續買超(選股條件 = []):

    選股條件.append(YOY > 50)
    選股條件.append((近3月營收年增率 > 近12月營收年增率) & (近6月營收年增率 > 近12月營收年增率))
    選股條件.append(近6月營收年增率 > 近6月營收年增率.shift(1))
    選股條件.append(R105_營業毛利率 > R105_營業毛利率.shift(2))
    選股條件.append(R105_營業毛利率 > R105_營業毛利率.shift(3))
    
    選股條件.append((當日成交值排行 < 30).rolling(10).sum() >= 2)
    選股條件.append((投信買賣金額排行 < 20).rolling(10).sum() >= 2)
    選股條件.append((外資買賣金額排行 < 40).rolling(10).sum() >= 2)
    
    選股條件.append(漲幅1 > 3)
    
    return 選股條件


stock_ids = (收盤價.columns & df產業類別.index & 股本.columns)
my_strategy = 法人連續買超([])
pick_stocks(datetime.date(2020, 11, 6), stock_ids, my_strategy, 2)

In [ ]:
import datetime
import pandas as pd
import os
import matplotlib.pyplot as plt
from finlab.data import Data
from finlab.plot_candles import plot_candles

短多家數 = rising_20.fillna(False).apply(pd.Series.value_counts, axis=1)


# 讀取 goodinfo 的電子產業

In [ ]:
import io
import json
import requests
import datetime
import pandas as pd
from io import StringIO
import os

stock_all = pd.DataFrame()
group_path = 'stock_group'

for stock_group in os.listdir(group_path):
    
    if stock_group[-4:] != '.csv':
        continue

    try:
        stocks = pd.read_csv(os.path.join(group_path, stock_group), encoding='utf8')
        stocks['goodinfo電子產業'] = stock_group[:-4]
    except:
        print(stock_group)
        continue
        
    stock_all = stock_all.append(stocks, ignore_index=True)
        
stock_all = stock_all.rename(columns={'Stock ID': 'stock_id', 'Stock Name':'name'})
stock_all = stock_all.astype(str).set_index('stock_id').sort_index()
stock_all.to_pickle(os.path.join(group_path, 'stock_all_group.pkl'))

In [ ]:
date = datetime.date(2018,1,16)
漲幅 = percentage(收盤價, 收盤價.shift(1))

conditions = []
conditions.append(漲幅.rolling(5).max() >= 9)
conditions.append(成交量.rolling(20).min() > 500)

satisfy = pd.Series(index=收盤價.columns, data=True, name=date)
for condition in conditions:
    satisfy &= condition.loc[date:].iloc[0]
    
satisfy_stocks = satisfy[satisfy]
強勢股清單 = satisfy_stocks.index & 產業分類idx
產業族群計數 = 產業分類.loc[強勢股清單, '細產業'].value_counts()
強勢產業族群 = 產業族群計數[產業族群計數 >= 3]
強勢產業族群


In [ ]:
indeustry_table = 產業分類[產業分類['細產業'] != 產業分類['產業別']].copy()

營收爆發=[]
for i in 強勢產業族群.index.to_list():
    
    sid = pick_industry_stocks('細產業', i)
    營收 = (YOY.loc[date:, sid.index].iloc[0] > 100).value_counts()
    if True in 營收:
        營收爆發.append(i)

營收爆發

In [ ]:
start_date = datetime.date(2020, 9, 22)
end_date = datetime.date(2020, 10, 7)
# end_date = datetime.datetime.today()
backtest_dates = 收盤價.loc[start_date:end_date, :].index.tolist()

漲幅 = percentage(收盤價, 收盤價.shift(1))
conditions = []

突破 = (收盤價 > upper) & (漲幅 > 3)
conditions.append(突破.rolling(5).sum() >= 1)
conditions.append(成交量.rolling(20).min() > 500)

加權月乖離 = percentage(加權收盤價, 加權月均線)

強勢產業族群表 = []
營收爆發表=[]
for date in backtest_dates:
    
    satisfy = pd.Series(index=收盤價.columns, data=True, name=date)
    for condition in conditions:
        satisfy &= condition.loc[date:].iloc[0]
        
    satisfy_stocks = satisfy[satisfy]
    強勢股清單 = satisfy_stocks.index & 產業分類idx
    產業族群計數 = 產業分類.loc[強勢股清單, '細產業'].value_counts()
    強勢產業族群 = 產業族群計數[產業族群計數 >= 3]
    強勢產業族群表.append(強勢產業族群.index.to_list())
    
    營收爆發=[]
    for i in 強勢產業族群.index.to_list():
        sid = pick_industry_stocks('細產業', i)
        營收 = (YOY.loc[:date, sid.index].iloc[-1] > 50).value_counts()
        if True in 營收:
            if 營收[True] >= 2:
                營收爆發.append(i)
            
    營收爆發表.append(營收爆發)
        
    
df強勢產業族群 = pd.DataFrame(data={'date':backtest_dates, 
                              '產業': 強勢產業族群表, 
                              '營收爆發' : 營收爆發表,
                              '月乖離': 加權月乖離.loc[start_date:end_date]})
df強勢產業族群.set_index('date', inplace=True)
df強勢產業族群

In [ ]:
stocks = pick_industry_stocks('細產業', '消費性IC')

idx = 收盤價.columns & stocks.index
date = datetime.date(2020,8,21)
a = 漲幅.loc[date, idx]
b = 收盤價.loc[date, idx].rank(ascending=False)
c = 收盤價.loc[date, idx]
d = pd.concat({'漲幅': a,'收盤價':c, '排行': b}, axis=1).sort_values('漲幅').dropna(how='all')
d

In [ ]:
import datetime
import pandas as pd
import os
import matplotlib.pyplot as plt
from finlab.data import Data
from finlab.plot_candles import plot_candles

stocks = pick_industry_stocks('細產業', '消費性IC')
date = datetime.date(2020, 9, 22)
# print(stocks.index)
外資累積買賣超 = 外資買賣超.rolling(3).sum()
投信累積買賣超 = 投信買賣超.rolling(3).sum()
法人累積買賣超 = 法人買賣超.rolling(5).sum()
最小成交量 = 成交量.rolling(20).min()

idx = 收盤價.columns & stocks.index

a = 漲幅.loc[date, idx]
b = 收盤價.loc[date, idx].rank(ascending=False)
c = 收盤價.loc[date, idx]
d = pd.concat({'漲幅': a,'收盤價':c, '排行': b}, axis=1).sort_values('漲幅').dropna(how='all')


# for sid in d.index[-5:]:
for sid in stocks.index.drop_duplicates():
    
    if len(sid) > 4:
        continue
    
#     if (最小成交量.loc[date:, sid].iloc[0] < 500 or
#         YOY.loc[:date, sid].iloc[-1] < 0):
#         continue
        
#     if (最小成交量.loc[date:, sid].iloc[0] < 500 or
#         YOY.loc[:date, sid].iloc[-1] < 0 or
#         MOM.loc[:date, sid].iloc[-1] < 0):
#         continue

    if (法人累積買賣超.loc[date:, sid].iloc[0] < 0 or 
        最小成交量.loc[date:, sid].iloc[0] < 500):
        continue

    pricing = pd.DataFrame({'open': 開盤價[sid], 
                            'close': 收盤價[sid], 
                            'high': 最高價[sid], 
                            'low': 最低價[sid], 
                            'volume': 成交量[sid]})

    hint = '{}, {}, YOY:{}, MOM:{}, Fund:{}'.format(date.strftime('%Y-%m-%d'), sid, 
                                          YOY.loc[:date, sid].iloc[-1],
                                          MOM.loc[:date, sid].iloc[-1],
                                          int(法人累積買賣超.loc[date:, sid].iloc[0]))

    plot_candles(start_time=收盤價.loc[:date].index[-59], 
                 end_time=收盤價.loc[date:].index[0], 
                 pricing=pricing, volume_bars=True,
                 title=hint, 
                 overlays=[upper[sid], ma20[sid], lower[sid], ma60[sid], ma5[sid]])
    
    plt.rcParams["figure.figsize"] = (20,8)
    plt.show()

In [ ]:
加權5ma_rate = percentage(加權周均線, 加權周均線.shift(1))
急殺 = 加權5ma_rate < -0.75

止跌 = (加權5ma_rate < 0) & (加權收盤價.shift(1) < 加權周均線.shift(1)) & (加權收盤價 >= 加權周均線)

急殺 = 急殺.replace(True, 1)
止跌 = 止跌.replace(True, 1)

In [ ]:
import datetime

yday=datetime.date.today() - datetime.timedelta(days=1)

ax = 強勢股[True].loc[sd:ed].plot()
ax.set_xlim([sd-datetime.timedelta(days=1), ed+datetime.timedelta(days=1)])
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d')) 
ax.xaxis.set_major_locator(mdates.DayLocator(interval=3))
# ax.set_xmargin(2)
plt.gcf().autofmt_xdate(rotation=90) 